In [ ]:
# ! pip3 install pandas matplotlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

ALPHA = 0.05
EPSILON = 0.30
COST_WEIGHTS = [[0.66, 0.39],  # high efficient thermal generator
                [0.69, 0.39],  # medium efficient thermal generator
                [0.79, 0.39],  # low efficient thermal generator
                [0, 0],        # wind generator
                ]
SUPPLY_QUOTAS = [   0.334, 
                    0.333, 
                    0.333, 
                    1.0,
                ]

PRICE_MUTIPLIERS = [1.1, 1.3, 2.0]
PRICE_LABELS = ['Low', 'Medium', 'High']
GNERATOR_NAMES = ['thermal #1', 'thermal #2', 'thermal #3', 'wind generator']


data = pd.read_csv('data/Electricity_Data.csv')
col_names = ['DemandAggregatedForecast', 'WindAggregatedForecast', 'MULTI_FUEL_-24', 'GAS_-24', 'GasPrice','CarbonPrice','EURPrices']
data = data[col_names]
print(len(data))
data.head()


In [ ]:
train_num =24*500 # each bar is 1 Hour, 24 Hours = 1 Day
train_data = data.iloc[:train_num, :]
test_data = data.iloc[train_num:, :]

train_data.shape, test_data.shape

In [ ]:
# supply_multiplier 1.0: the original supply capacity of the thermal & wind power; 3.0: 3 times the original supply capacity
supply_multiplier = [1.0, 4.0] #

# plot demand, original_supply_capacity, 4*original_supply_capacity 
SupplyAggregated = data.apply(lambda x: (x[1]-x[2]-x[3]), axis=1)

plt.figure(figsize=(18, 10))
plt.plot(data['DemandAggregatedForecast'], label='DemandAggregated')
for sm in supply_multiplier:
    plt.plot(SupplyAggregated*sm, label=f'SupplyAggregated:{sm}x')
plt.legend()
plt.show()

In [ ]:

class Gencos:
    def __init__(self, id, cost_weights, epsilon=0.3, alpha=0.05, supply_quota=1., wind_state_splits=[]):
        self.id = id
        self.cost_weights = cost_weights # [gas_cost_weight, carbon_cost_weight]
        self.epsilon = epsilon
        self.alpha = alpha
        self.bid_price_factor = np.array(PRICE_MUTIPLIERS)
        self.supply_quota = supply_quota
        self.wind_state_splits = wind_state_splits
        self.Q = np.zeros((len(wind_state_splits)+1, len(self.bid_price_factor)))
        self.Q_history = [self.Q.copy()]

    def get_wind_state(self, wind_forecast):
        if len(self.wind_state_splits) ==0:
            return 0 
        if wind_forecast <= self.wind_state_splits[0]:
            return 0
        if wind_forecast > self.wind_state_splits[-1]:
            return len(self.wind_state_splits)

        for i in range(len(self.wind_state_splits)-1):
            if wind_forecast > self.wind_state_splits[i] and wind_forecast <= self.wind_state_splits[i+1]:
                return i+1

    def bid(self, multi_fuel, gas, wind_aggregated_forcast, gas_price, carbon_price, mode='train'):
        self.wind_state = self.get_wind_state(wind_aggregated_forcast)
        
        # get a bid option
        R = np.random.random()
        if mode == 'train' and R <= self.epsilon:
            # bid with a random price
            self.bid_index = np.random.randint(0,len(self.bid_price_factor))
        else:
            # bid with max_Q's price
            self.bid_index = self.Q[self.wind_state,:].argmax() 

        # compute bid price
        self.cost = self.cost_weights[0]*gas_price + self.cost_weights[1]*carbon_price
        if self.cost == 0:
            bid_price = self.bid_price_factor[self.bid_index]*(0.69*gas_price + 0.39*carbon_price)
        else:
            bid_price = self.bid_price_factor[self.bid_index]*self.cost

        # compute the capacity of the generator
        # wind generator's capacity
        if sum(self.cost_weights) == 0:
            bid_capacity = wind_aggregated_forcast*self.supply_quota
        # thermal generator's capacity
        else:
            bid_capacity = -1.0*(multi_fuel + gas)*self.supply_quota

        return bid_price, bid_capacity, self.cost, self.wind_state, self.bid_index, self.id

    def update_Q(self, success_bid_capacity, clean_price, bid_index, gencos_id):
        if self.id == gencos_id:
            # Qij ←(1 − αit)Qij + αit(ri)
            self.Q[bid_index] = (1 - self.alpha)*self.Q[bid_index] + self.alpha*success_bid_capacity*(clean_price-self.cost)
            self.Q_history.append(self.Q.copy())
    
    def update_Q_batch(self, bid_results_batch):
        bid_results_df = pd.DataFrame(bid_results_batch)
        bid_results_df.columns = ['price', 'capacity', 'cost', 'wind_state', 'bid_index', 'id']

        for s in range(len( self.wind_state_splits)+1):
            for i in range(len(self.bid_price_factor)):
                bids = bid_results_df[(bid_results_df['id']== self.id) & (bid_results_df['wind_state']== s) & (bid_results_df['bid_index']== i)]
                # Qij ←(1 − αit)Qij + αit(ri)
                if len(bids) > 0:
                    bids_reward = ((bids['price'] - bids['cost'])*bids['capacity']).mean()
                    self.Q[s, i] = (1 - self.alpha)*self.Q[s,i] + self.alpha*bids_reward
        self.Q_history.append(self.Q.copy())

class ISO:
    def __init__(self, gencos):
        self.gencos = gencos
        self.mode = 'train'
        self.clean_prices = []
    
    def get_bid_list(self, multi_fuel_24, gas_24, wind_aggregated_forcast, gas_price, carbon_price):
        bid_list = [g.bid(multi_fuel_24, gas_24, wind_aggregated_forcast, gas_price, carbon_price, mode=self.mode) for g in self.gencos]
        return bid_list

    def clean_bids(self, bid_list, demond, price_rule='uniform', ration_policy='random'):

       
        
        total_volume = 0
        clear_price = 0
        bid_result = []

        # sort by price ascending
        bid_list.sort()
        # convert list to np array
        bid_array = np.array(bid_list)
        # grop the bids by bid price
        bid_group = np.split(bid_array, np.unique(bid_array[:, 0], return_index=True)[1][1:])

        # convert bid groups into bid_volume and genco blocks (bid_price, volume_sum)
        price_volume_block = []
        for g in bid_group:
            price_volume_block.append((g[0,0], g[:,1].sum()))

        # clean the bids
        # demond >= supply: clean_price = highest_bid_price
        if sum([pv[1] for pv in price_volume_block])<= demond:
            clean_price = price_volume_block[-1][0]
            bid_result = bid_list
        
        # demond < supply
        else:
            for price, volume in price_volume_block:
                if (total_volume + volume) < demond:
                    total_volume += volume
                else:
                    # get the clean_price
                    clean_price = price
                    # get the left volume for clear_price bids
                    left_volume = demond - total_volume

                    success_bids = [b for b in bid_list if b[0]< clean_price]
                    clean_bids = [b for b in bid_list if b[0]== clean_price]
                    # get the fail bids and set volume to 0
                    fail_bids = [(b[0], 0, b[2], b[3], b[4], b[5]) for b in bid_list if b[0]> clean_price]


                    # rationing policy
                    ration_bids = []

                    # rationing policy = equal
                    if ration_policy == 'equal':
                        volume_ratio = left_volume/volume
                        ration_bids = [(b[0], b[1]*volume_ratio, b[2], b[3], b[4], b[5]) for b in clean_bids]

                    # rationing policy = random
                    if ration_policy == 'random':
                        while left_volume > 0:
                            idx = np.random.randint(len(clean_bids))
                            select_bid = clean_bids[idx]
                            clean_bids.remove(select_bid)
                            if left_volume > select_bid[1]:
                                ration_bids.append(select_bid)
                                left_volume -= select_bid[1]
                            else:
                                select_bid = (select_bid[0], left_volume, select_bid[2], select_bid[3], select_bid[4], select_bid[5])
                                ration_bids.append(select_bid)

                                clean_bids = [(b[0], 0, b[2], b[3], b[4],b[5]) for b in clean_bids]
                                fail_bids.extend(clean_bids)
                                break

                    bid_result.extend(success_bids)
                    bid_result.extend(ration_bids)
                    bid_result.extend(fail_bids)
                    break

        

        # pricing rule
        if price_rule == 'uniform':
            bid_result = [(clean_price, b[1], b[2], b[3], b[4], b[5]) for b in bid_result]

        return bid_result, clean_price

    def run(self, data, price_rule='uniform', ration_policy='random', batch_size=24, epoch = 1 ):
        self.clean_prices = []

        data_len = len(data)
        batch_num = math.ceil(data_len/batch_size)

        for e in range(epoch):
            for bn in range(batch_num):
                bid_result_batch = []
                for  i in range(batch_size):
                    demond, wind_aggregated_forcast, multi_fuel_24, gas_24, gas_price, carbon_price, _  = data.iloc[i+bn*batch_size,:]
                    bid_list = self.get_bid_list(multi_fuel_24, gas_24, wind_aggregated_forcast, gas_price, carbon_price)
                    bid_result, clean_price = self.clean_bids(bid_list, demond,  price_rule, ration_policy)
                    self.clean_prices.append(clean_price)
                    bid_result_batch.extend(bid_result)
                    
                    print(f'batch:#{bn}      bid round: #{i+1}      run_mode: {self.mode}') 
                    print('demond:', demond)
                    print('bid list(bid_price, supply_capacity, gencos_id):\n', bid_list)
                    print('clean price:', clean_price)
                    print('bid_result(price, accepted_capacity, gencos_id):\n', bid_result)
                    print('#'*160, '\n')
                

                # update Q-value
                if self.mode =='train':
                    print('update the Q-values')
                    for g in self.gencos:
                        g.update_Q_batch(bid_result_batch)



## Case 1: two different price rules

In [ ]:
test_gencos = []
price_rules = ['pay_as_bid', 'uniform']
for pr in price_rules:
    # initialize thermal & wind generators
    gencos = [Gencos(i, COST_WEIGHTS[i], EPSILON, ALPHA, SUPPLY_QUOTAS[i]) for i in range(4)]

    test_gencos.append(gencos)

    # initialize power market ISO
    power_market_iso = ISO(gencos)

    # train the generators
    power_market_iso.mode = 'train'
    power_market_iso.run(train_data, price_rule=pr, ration_policy='random', batch_size=24, epoch = 2)

In [ ]:
figure = plt.figure(figsize=(20, 10))

test_num = len(test_gencos)
gencos_num = len(test_gencos[0])
for i in range(test_num):
    for j in range(gencos_num):
        q_data = np.array(test_gencos[i][j].Q_history)
        plt.subplot(test_num, gencos_num, j+1+i*gencos_num)
        plt.plot(q_data[:,0,:], label=[f'bid_price={p}' for p in PRICE_LABELS])
        plt.legend()
        plt.title(f'{GNERATOR_NAMES[j]}  ({price_rules[i]})')
        plt.xlabel('bid iteration')
        plt.ylabel('Q value')

## Case 2: two different supply quantity

In [ ]:
test_gencos = []
supply_multiplier = [1.0, 4.0]
for sm in supply_multiplier:
    # initialize thermal & wind generators
    gencos = [Gencos(i, COST_WEIGHTS[i], EPSILON, ALPHA, SUPPLY_QUOTAS[i]*sm) for i in range(4)]

    test_gencos.append(gencos)

    # initialize power market ISO
    power_market_iso = ISO(gencos)

    # train the generators
    power_market_iso.mode = 'train'
    power_market_iso.run(train_data, price_rule='pay_as_bid',ration_policy='random', batch_size=24, epoch = 2)

In [ ]:
figure = plt.figure(figsize=(20, 10))

test_num = len(test_gencos)
gencos_num = len(test_gencos[0])
for i in range(test_num):
    for j in range(gencos_num):
        q_data = np.array(test_gencos[i][j].Q_history)
        plt.subplot(test_num, gencos_num, j+1+i*gencos_num)
        plt.plot(q_data[:,0,:], label=[f'bid_price={p}' for p in PRICE_LABELS])
        plt.legend()
        plt.title(f'{GNERATOR_NAMES[j]}  (supply:{supply_multiplier[i]}x)')
        plt.xlabel('bid iteration')
        plt.ylabel('Q value')

## Case 3: experiment with three different wind states

In [ ]:
wind_state_divier = data['WindAggregatedForecast'].quantile([0.5, 0.8]).values
wind_state_divier


In [ ]:
plt.plot(data['WindAggregatedForecast'], label='WindAggregatedForecast')
plt.plot(np.linspace(0,33000, 100), np.multiply(np.ones((100,2)), wind_state_divier), label = ['low', 'high'])
plt.legend()

In [ ]:

# initialize thermal & wind generators
gencos = [Gencos(i, COST_WEIGHTS[i], EPSILON, ALPHA, SUPPLY_QUOTAS[i],wind_state_splits=wind_state_divier) for i in range(4)]

# initialize power market ISO
power_market_iso = ISO(gencos)

# train the generators
power_market_iso.mode = 'train'
power_market_iso.run(train_data, price_rule='pay_as_bid', ration_policy='random', epoch=2)

In [ ]:
figure = plt.figure(figsize=(20, 20))

generator_names = ['thermal generator #1', 'thermal generator #2', 'thermal generator #3', 'wind generator']
state_names = ['low','medium', 'high']

state_num = len(state_names)
gencos_num = len(gencos)
n = 1
for i in range(gencos_num):
    for j in range(state_num):       
        q_data = np.array(gencos[i].Q_history)
        plt.subplot( gencos_num, state_num, n)
        plt.plot(q_data[:,j,:], label=[f'bid_price={p}' for p in PRICE_LABELS])
        plt.legend()
        plt.title(f'{generator_names[i]}  wind state:{state_names[j]}')
        plt.xlabel('bid iteration')
        plt.ylabel('Q value')
        n += 1